### Импорт

In [71]:
from natasha import (
    Segmenter,
    MorphVocab,
    PER, 
    NewsEmbedding,
    Doc,
    NewsMorphTagger,
    NewsNERTagger,
    NamesExtractor
)
import nltk
from nltk.corpus import stopwords
from string import punctuation
import requests
import re
from requests.adapters import HTTPAdapter, Retry
import time

emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

session = requests.Session()
retries = Retry(
    total=5,              
    backoff_factor=0.5,   
    status_forcelist=[429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://", adapter)


### Предобработка

In [72]:
noise = stopwords.words("russian") + list(punctuation) + ['—', '«', '»', '\'\'']

characters = [
    "Татарский",
    "Морковин",
    "Гусейн",
    "Сергей",
    "Лена",
    "Азадовский",
    "Слава",
    "Зайцев",
    "Вова", 
    "Морковина",
    "Вавилен",
    "Дмитрий",
    "Пугин",
    "Дмитрий Пугин",
    "Сергей Морковин",
    "Гиреев",
    "Андрей",
    "Андрей Гиреев",
    "Леонид",
    "Азадовский",
    "Леонид Азадовский",
    "Вовчик",
    "Малой",
    "Вовчик Малой",
    "Саша Бло",
    "Саша",
    "Леша Чикунов",
    "Эдик",
    "Григорий",
    "Владимир Ханин",
    "Ханин",
    "Владимир",
    "Малюта",
    "Аркаша",
    "Алла",
    "Семен",
    "Фарсейкин",
    "Фарсук Сейфуль-Фарсейкин",
    "Фарсук",
    "Фарсук Карлович",
    "Манька"]   

# def cleaner(text):
#     tokens = nltk.word_tokenize(text)
#     tokens = [token for token in tokens if token not in noise]
#     clean_text = " ".join(tokens)
#     return clean_text

text = ""

with open("Pelevin_Generation_p.txt", 'r', encoding="UTF-8") as f:
    for line in f:
        line = re.sub(r'^"', '', line)
        line = re.sub(r'"\s*$', ' ', line)
        line = re.sub(r'[\t\n\r]+', ' ', line)
        text += line

doc = Doc(text)

In [73]:
#Токенизация 
doc.segment(segmenter)

#Морфологизация
doc.tag_morph(morph_tagger)

#Лемматизация
for token in doc.tokens:
    token.lemmatize(morph_vocab)

doc.tokens[:5]

[DocToken(stop=7, text='Leonard', pos='X', feats=<Yes>, lemma='leonard'),
 DocToken(start=8, stop=13, text='Cohen', pos='X', feats=<Yes>, lemma='cohen'),
 DocToken(start=14, stop=22, text='Когда-то', pos='ADV', feats=<Pos>, lemma='когда-то'),
 DocToken(start=23, stop=24, text='в', pos='ADP', lemma='в'),
 DocToken(start=25, stop=31, text='России', pos='PROPN', feats=<Inan,Loc,Fem,Sing>, lemma='россия')]

#### Функции

In [74]:
def per_to_correct_per(per):
    r = session.get(
    "https://ru.wikipedia.org/w/api.php",
    params={
        "action": "query",
        "list": "search",
        "srsearch": per,      
        "format": "json"
        }
    )
    try: 
        data = r.json()
    except:
        return ""
    if not data["query"]["search"]:
        return ""
    data = r.json()
    return data["query"]["search"][0]["title"]

def get_per_id(per):
    url = f"https://ru.wikipedia.org/w/api.php"  
    r = session.get(url, params={
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "titles": per
    }) 
    try:
        data = r.json()["query"]
    except:
        return None
    data = r.json()["query"]
    pages = dict(data["pages"])
    for page_id, info in pages.items(): 
        if page_id != "-1": 
            if "wikibase_item" in info.get("pageprops", {}):
                return info["pageprops"]["wikibase_item"] 
    return None

def get_occupation_id(list_id):
    dict_id_id = {}
    str_list_id = "|".join(list_id)
    url = f"https://www.wikidata.org/w/api.php"
    params = {
       "action": "wbgetentities",
       "ids": str_list_id,
       "props": "claims|labels",
       "languages": "ru",
       "format": "json"
    }
    r = session.get(url, params=params)
    try:
        data = r.json()["entities"]
    except:
        return {}
    data = r.json()["entities"]
    for qid, entity_data in data.items():
        claims = entity_data.get("claims", {})
        occupation_claims = claims.get("P106", [])
        list_occ_id = list()
        for claim in occupation_claims:
            try:
                occ_id = claim["mainsnak"]["datavalue"]["value"]["id"]
                list_occ_id.append(occ_id)
            except:
                pass
        dict_id_id[qid] = list_occ_id
    return dict_id_id

def get_occupation(occ_id_list):
    if not occ_id_list:
        return {}
    occ_id_str = "|".join(occ_id_list)
    params = {
       "action": "wbgetentities",
       "ids": occ_id_str,
       "props": "labels",
       "languages": "ru",
       "format": "json"
    }
    url = "https://www.wikidata.org/w/api.php"
    r = session.get(url, params=params)
    data = r.json()
    res = []
    if "entities" not in data:
        return res
    for value in data["entities"].values():
        res.append(value.get("labels", {}).get("ru", {}).get("value", ""))
    return res
    


def char_filter(doc):
    filtered_spans = []
    for span in doc.spans:
        skip_span = False
        for token in span.tokens:
            lemma = token.lemma.capitalize()
            if lemma in characters:
                skip_span = True
                break
        if not skip_span:
            filtered_spans.append(span)
    return filtered_spans

### NER

In [76]:
doc.tag_ner(ner_tagger)


doc.spans = char_filter(doc) 
# for span in doc.spans:
#     print(span.tokens)
    # print("token:", span.text, "type:", span.type)
# doc.ner.print()

Получение списка личностей, организаций и локаций

In [78]:
list_per = list()
set_per = set()
list_loc = list()
list_org = list()
for span in doc.spans:
    if (span.type == "PER"):
        for token in span.tokens:
            list_per.append(token.lemma)
            set_per.add(token.lemma)
    elif (span.type == "LOC"):
        for token in span.tokens:
            list_loc.append(token.lemma)
    else:
        for token in span.tokens:
            list_org.append(token.lemma)
print(len(set_per))
print(*list_per[:10])

343
брежнев вильям бойд василий аксенов аксенов ленин борис пастернак пастернак


### Парсинг

In [67]:
occupations_id = {}
occupations = {}
id_list = []
dict_id_per = {}
for i in range(len(list_per)):
    per = list_per[i]
    if per not in occupations.keys():
        correct_per = per_to_correct_per(per)
        time.sleep(0.2)
        qid = get_per_id(correct_per)
        time.sleep(0.2)
        if qid != None:
            id_list.append(qid)
            dict_id_per[qid] = per
        if (i + 1) % 45 == 0 or (i + 1) == len(list_per):
            dict_id_id = get_occupation_id(id_list)
            time.sleep(0.2)
            for key, value  in dict_id_id.items():
                occ_id_list = set()
                occupations_id[dict_id_per[key]] = value
                for occ_id in value:
                    occ_id_list.add(occ_id)
                occ_list = get_occupation(occ_id_list)
                occupations[dict_id_per[key]] = occ_list
            id_list = []
            dict_id_per = {}
    else:
        pass

for item, value in occupations.items():
    print(item, ':', *value)

брежнев : землемер инженер военнослужащий политик
вильям :
бойд :
василий :
аксенов : учитель романист поэт сценарист писатель врач
ленин : практикующий юрист Вождь мирового пролетариата политический теоретик политик экономист философ журналист писатель революционер
борис :
пастернак : романист пианист поэт переводчик драматург прозаик писатель
марина :
цветаев : археолог историк
сталин : публицист лингвист политик государственный деятель революционер
троцкий : дипломат автобиограф историк политик военнослужащий философ журналист писатель революционер
достоевский : эссеист публицист романист писатель поэт переводчик новеллист философ журналист прозаик биограф
геббельс : романист демагог автобиограф автор дневника политик сценарист журналист пропагандист писатель
дракула : некромант солдат алхимик государственный деятель волшебник
усложнять :
нил :
шиза :
ржевский :
рональд : телеактёр профсоюзный деятель офицер ведущий игрового шоу автобиограф автор дневника актёр политик киноактёр акт